# 04 | dbt Marts and Gold Export

This notebook runs dbt models and docs generation, then exports marts schema tables from DuckDB into Gold Parquet outputs.

In [ ]:
# Parameters
source = "fivethirtyeight"
dataset = "recent_grads,bechdel_movies"
run_date = "2026-02-22"
force_refresh = False

In [ ]:
import sys
from pathlib import Path

ROOT_DIR = Path.cwd()
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

In [ ]:
import os
import subprocess
import sys
import time
from pathlib import Path

import duckdb
import pandas as pd

from src.common.io import update_pipeline_metrics, update_stage_metrics
from src.common.paths import DBT_PROJECT_DIR, GOLD_DIR, SILVER_DIR, WAREHOUSE_DB_PATH

os.environ['SILVER_RECENT_GRADS_PATH'] = str((SILVER_DIR / 'recent_grads' / 'data.parquet').resolve())
os.environ['SILVER_BECHDEL_MOVIES_PATH'] = str((SILVER_DIR / 'bechdel_movies' / 'data.parquet').resolve())

stage_start = time.perf_counter()
dbt_cli = Path(sys.executable).with_name('dbt')

subprocess.run(
    [
        str(dbt_cli),
        'run',
        '--project-dir',
        str(DBT_PROJECT_DIR),
        '--profiles-dir',
        str(DBT_PROJECT_DIR),
    ],
    check=True,
)

subprocess.run(
    [
        str(dbt_cli),
        'docs',
        'generate',
        '--project-dir',
        str(DBT_PROJECT_DIR),
        '--profiles-dir',
        str(DBT_PROJECT_DIR),
    ],
    check=True,
)

connection = duckdb.connect(str(WAREHOUSE_DB_PATH))
query = "select table_name from information_schema.tables where table_schema = 'marts' order by table_name"
tables = [row[0] for row in connection.execute(query).fetchall()]

rows_gold = 0
summary_rows = []

for table_name in tables:
    output_path = GOLD_DIR / table_name / 'data.parquet'
    output_path.parent.mkdir(parents=True, exist_ok=True)
    if output_path.exists():
        output_path.unlink()

    connection.execute(
        f"COPY (SELECT * FROM marts.{table_name}) TO '{output_path.as_posix()}' (FORMAT PARQUET)"
    )
    row_count = int(connection.execute(f"SELECT COUNT(*) FROM marts.{table_name}").fetchone()[0])

    rows_gold += row_count
    summary_rows.append({'table': table_name, 'rows': row_count, 'gold_path': str(output_path)})

connection.close()
runtime_seconds = round(time.perf_counter() - stage_start, 2)

update_stage_metrics(
    'dbt_gold',
    {
        'runtime_seconds': runtime_seconds,
        'rows_gold': rows_gold,
        'tables': tables,
    },
)
update_pipeline_metrics({'rows_gold': rows_gold})

pd.DataFrame(summary_rows)